# Blog-Post about genome-scale metabolic modeling
This Blog-post is an introduction into genome-scale metabolic models (GSMMs). For this purpose the model iYli21 is used. How to interact with these models is implemented in the following notebook using a python framework for Constraint-based Reconstruction and Analysis (COBRA) called COBRApy.

At first the imports and setup is presented. Afterwards it is shown that a model consists of reactions, metabolies and genes. To get an intuition how this information could be used in practice there is a short example of how to skim through the components and build a table from this information. The model presentation is followed by a simulation. So the implementation of a flux balance analysis (FBA) is shown. The last part is about adding a pathway into the model. In this case the galactose pathway is added which is useful for the digestion of lignocellulosic biomass. 
 

## Table of contents:
### 1. Set up and imports
### 2. Presentation of the model (iYli21)
### 3. Simulation of the model (FBA)
### 4. Adding a Pathways (Galactose)

### 1. Setup working directory and imports
The next slot is for imports of python packages used and for the setup of the working directory

In [40]:
from cobra.io import read_sbml_model, write_sbml_model # reading and writing
from cobra import Reaction, Metabolite # for adding reactions and metabolites
from cobra import manipulation # for gene knockout
import pandas as pd # for dataframes
### Set up working dir:
wd = '../'
outdir = 'output/'
baseModel = 'iYli21_v1.xml'
extendedModelName = 'iYli21_blog.xml'

### 2. Presenting the model (iYli21)
#### Outline:
1. Reading a model
2. Overview over model (metabolites, reactions and genes)
3. Interactions of the components (From reaction to metabolites, from reaction to gene and from metabolite to reaction)
4. Groups of reactions

#### 1. Reading a model
The next slot is how you read a model in sbml format into a variable.

In [33]:
# The function read_sbml_model reads the model from the path to the model which is given as parameter
model2 = read_sbml_model(wd + baseModel)

#### 2. Overview over the model (Metabolites, Reactions and Genes)
The following slot shows how a model is represented in COBRApy and how to interact with them will be part of the following section.

We start with the compoenents: A model in COBRApy consists of 3 dictionarys and one name.
A reactions dictionary, a metabolites dictionary and a genes dictionary. They differ in the features stored.

In [34]:
# Show the types of the model components
print(type(model2.reactions))
print(type(model2.metabolites))
print(type(model2.genes))
# investigate number of metabolites, reactions and genes
modelName = model2.name
reacNum = len(model2.reactions)
metaboNum = len(model2.metabolites)
geneNum = len(model2.genes)

print('The {name} has {reacNum} reactions, {metaboNum} metabolites and {geneNum} genes'.format(name=modelName, reacNum=reacNum, metaboNum=metaboNum, geneNum=geneNum))


<class 'cobra.core.dictlist.DictList'>
<class 'cobra.core.dictlist.DictList'>
<class 'cobra.core.dictlist.DictList'>
The Genome-scale model of Yarrowia lipolytica W29 has 2285 reactions, 1868 metabolites and 1058 genes


In the following section the features of reactions, metabolites and genes will be shown. As you can see one metabolite is part of many reactions.
Each reaction has a name and an identifier as well as an Stoichiometry and a so called gene_protein_rule short GPR. 
The last two features are the lower and upper bound. 

For example the reaction glucose-6-phosphate isomerase has the id R326 and is responsible for the transfer of D-glucose-6-phosphate into D-fructose-6-phosphate. It is regulated from the gene YALI1F11049g and -1000.0 and 1000.0 are the lower and upper bounds, respectively. If the lower bound is negative it means it is reversible.

Each dictionary (reactions, metabolites and genes) has a function called get_by_id() which needs an id and returns the corresponding reaction, metabolite or gene, respectively. 

In [35]:
# Show the features of one reaction
model2.reactions.get_by_id('R326')

Reaction identifier,R326
Name,glucose-6-phosphate isomerase
Memory address,0x7f9e49899dc0
Stoichiometry,m296[C_cy] <=> m166[C_cy] D-glucose 6-phosphate_C6H13O9P <=> D-fructose 6-phosphate_C6H11O9P
GPR,YALI1F11049g
Lower bound,-1000.0
Upper bound,1000.0


The metabolite D-glucose 6-phosphate has the id m296[C_cy] and is part of the C_cy (an abbrevision for cytoplasma) compartment. The feature formula should be used for the chemical formula of the metabolite. In this model it was not used this could be a reason why memots identied problems. The last feature for metabolites is a list of reactions which it is part of. The following metabolite is part of 7 reactions.


In [36]:
# Show the features of one metabolite
model2.metabolites.get_by_id('m296[C_cy]')

Metabolite identifier,m296[C_cy]
Name,D-glucose 6-phosphate_C6H13O9P
Memory address,0x7f9e7bbb6af0
Formula,
Compartment,C_cy
In 7 reaction(s),"R325, R1988, R143, R638, R326, R387, R559"


A gene has dispite a name and an identifier two features. The functional feature shows if the genes is active (True) or knocked out (False). A list of reactions this gene regulates is the last feature.

In [37]:
# Show the features of one gene
model2.genes.get_by_id('YALI1F11049g')

Gene identifier,YALI1F11049g
Name,COBRAProtein360
Memory address,0x7f9e6bb0b940
Functional,True
In 1 reaction(s),R326


The summary of a model shows the used objective, which metabolites as well as reactions are used as uptake reactions and which metabolites are secreted. The corresponding fluxes are also mentioned.

In [38]:
# model summary 
print(model2.summary())

Objective
1.0 biomass_C = 0.2827037239904458

Uptake
------
 Metabolite Reaction    Flux  C-Number C-Flux
m1031[C_ex]    R1016    1.91         0  0.00%
 m511[C_ex]    R1070    2.43         0  0.00%
 m342[C_ex]    R1164   3.875         0  0.00%
m1339[C_ex]    R1287   5.057         0  0.00%
 m215[C_ex]    R1296 0.01286         0  0.00%
m1113[C_ex]    R1333 0.01919         0  0.00%

Secretion
---------
 Metabolite Reaction    Flux  C-Number C-Flux
m1176[C_ex]    R1030  -5.042         0  0.00%
 m214[C_ex]    R1364  -16.63         0  0.00%
m1401[C_cy]    R1373 -0.2827         0  0.00%



#### 3. Interactions of the model components (metabolites, reactions, genes)
In the following section I want to show how the parts of the model are connected.
For this purpose it is shown how to gain insights into the model by just building tables.

First, we want to build a table for all the reactions and its interacting components. 
We aim for the table to have the following columns: 

###### reactionId, reactionStoichiometry (formula of the reactions with the corresponding stoichiometric values), metaboliteIds, geneIds

In [48]:
# useful functions for interacting with metabolite, gene or reaction lists
def getIds(elementList):
    '''Returns the ids of an element list (list of genes, reactions or metabolites'''
    return [elem.id for elem in elementList]

def getNames(elementList):
    '''Returns the names of the elemts in the given list (list of genes, reactions or metabolites'''
    return [elem.name for elem in elementList]

In [55]:
allReactions = model2.reactions
reactionIds = []
reactionStoichimetries = []
metabolites = []
geneIds = []
# following code will use the function getIds() it is defined as usefunction at the beginning of this section
for reaction in allReactions:
    reactionIds.append(reaction.id)
    reactionStoichimetries.append(reaction.reaction)
    metabolites.append(getIds(reaction.metabolites))
    geneIds.append(getIds(reaction.genes))

# gathering information into a dataframe    
zipped = zip(reactionIds, reactionStoichimetries, metabolites, geneIds)
overviewDataframe = pd.DataFrame(zipped, columns=["reactionId", "reactionStoichimetry", "metaboliteIds", "geneIds"])
overviewDataframe.head()

,reactionId,reactionStoichimetry,metaboliteIds,geneIds
0,R1,m1[C_cy] + 2.0 m2[C_mi] --> 2.0 m3[C_mi] + m4[...,"[m1[C_cy], m2[C_mi], m3[C_mi], m4[C_cy]]","[YALI1E03870g, YALI1D11769g, YALI1C08548g]"
1,R2,2.0 m2[C_mi] + m5[C_mi] --> 2.0 m3[C_mi] + m6[...,"[m2[C_mi], m5[C_mi], m3[C_mi], m6[C_mi]]","[YALI1E03870g, YALI1D11769g]"
2,R3,2.0 m2[C_mi] + m7[C_cy] --> 2.0 m3[C_mi] + m4[...,"[m2[C_mi], m7[C_cy], m3[C_mi], m4[C_cy]]","[YALI1E03870g, YALI1D11769g]"
3,R4,m8[C_cy] --> m10[C_cy] + m11[C_cy] + m9[C_en],"[m8[C_cy], m9[C_en], m10[C_cy], m11[C_cy]]","[YALI1E25018g, YALI1C01944g]"
4,R5,m8[C_cy] --> m10[C_cy] + m11[C_cy] + m12[C_en],"[m8[C_cy], m10[C_cy], m11[C_cy], m12[C_en]]",[YALI1C19943g]


Now we can extend this table with the reaction name, the names of the metabolites and the genes. Why should we do this?
Because it is useful to have a searchable table of the reactions, metabolites and gene names and ids. With this information you will find specific metabolites or reactions much faster.
For example, if you try to integrate a new pathway it is very useful to know which of the corresponding metabolites are already part of your model. 
So you will have a tabe which includes names one can query on one hand and the corresponding id's to interact with these components on the other hand.
We will extend the aforementioned table and add the following columns:

###### reactionName, metaboliteName, geneName

In [60]:
reactionNames = []
metabolieNames = []
geneNames = []

# following code will use the function getNames() it is defined as usefunction at the beginning of this section
for reaction in allReactions:
    reactionNames.append(reaction.name)
    metabolieNames.append(getNames(reaction.metabolites))
    geneNames.append(getNames(reaction.genes))

# insert information into the table/dataframe
overviewDataframe["reactionNames"] = reactionNames
overviewDataframe["metaboliteNames"] = metabolieNames
overviewDataframe["geneNames"] = geneNames
# reorder the columns
overviewDataframe = overviewDataframe[["reactionId", "reactionNames", "reactionStoichimetry", "metaboliteIds", "metaboliteNames", "geneIds", "geneNames"]]
overviewDataframe.to_csv(outdir + "overviewDataframe.csv", index=False)
overviewDataframe.head()

,reactionId,reactionNames,reactionStoichimetry,metaboliteIds,metaboliteNames,geneIds,geneNames
0,R1,(R)-lactate:ferricytochrome-c 2-oxidoreductase,m1[C_cy] + 2.0 m2[C_mi] --> 2.0 m3[C_mi] + m4[...,"[m1[C_cy], m2[C_mi], m3[C_mi], m4[C_cy]]","[(R)-lactate_C3H6O3, ferricytochrome c_, ferro...","[YALI1E03870g, YALI1D11769g, YALI1C08548g]","[COBRAProtein3, COBRAProtein2, COBRAProtein1]"
1,R2,(R)-lactate:ferricytochrome-c 2-oxidoreductase,2.0 m2[C_mi] + m5[C_mi] --> 2.0 m3[C_mi] + m6[...,"[m2[C_mi], m5[C_mi], m3[C_mi], m6[C_mi]]","[ferricytochrome c_, (R)-lactate_C3H6O3, ferro...","[YALI1E03870g, YALI1D11769g]","[COBRAProtein3, COBRAProtein2]"
2,R3,(S)-lactate:ferricytochrome-c 2-oxidoreductase,2.0 m2[C_mi] + m7[C_cy] --> 2.0 m3[C_mi] + m4[...,"[m2[C_mi], m7[C_cy], m3[C_mi], m4[C_cy]]","[ferricytochrome c_, (S)-lactate_C3H6O3, ferro...","[YALI1E03870g, YALI1D11769g]","[COBRAProtein3, COBRAProtein2]"
3,R4,"1,3-beta-glucan synthase",m8[C_cy] --> m10[C_cy] + m11[C_cy] + m9[C_en],"[m8[C_cy], m9[C_en], m10[C_cy], m11[C_cy]]","[UDP-D-glucose_C15H22N2O17P2, (1-3)-beta-D-glu...","[YALI1E25018g, YALI1C01944g]","[COBRAProtein5, COBRAProtein4]"
4,R5,(1-6)-beta-glucan synthase,m8[C_cy] --> m10[C_cy] + m11[C_cy] + m12[C_en],"[m8[C_cy], m10[C_cy], m11[C_cy], m12[C_en]]","[UDP-D-glucose_C15H22N2O17P2, H+_p+1, UDP_C9H1...",[YALI1C19943g],[COBRAProtein6]


It is not only possible to get information about the metabolites from the reaction but also to find reaction information from the metabolies, as you can see in the following code block.


We want to find the metabolite glucose and show the reactions it is part of.


First of all we search in the table we just created and find the ids for D-glucose. There are different types of D-glucose but we are interested in D-glucose from the cytoplasm. For this model goes that the component of the model is shown in the id in brackets, in this case C_cy stands for cytoplasm. But this does not need to be the case in general.

In [77]:
dGlucose = model2.metabolites.get_by_id('m295[C_cy]')
dgluReactions = getIds(dGlucose.reactions)
print("Number of reactions d-glucose is part of: ", len(dgluReactions))
reactionNames = getNames(dGlucose.reactions)
print(f'D-gluose is part of the reactions with the following id: {dgluReactions} and the following names: {reactionNames}')

Number of reactions d-glucose is part of:  8
D-gluose is part of the reactions with the following id: ['R851', 'R1930', 'R323', 'R2059', 'R2184', 'R1138', 'R142', 'R387'] and the following names: ['glucose transport', 'cellobiose glucohydrolase', 'glucan 1,4-alpha-glucosidase', 'glucose 1-dehydrogenase', '1,4-alpha-D-Glucan glucohydrolase', 'glucose transport, vacuolar', 'alpha,alpha-trehalase', 'hexokinase (D-glucose:ATP)']


#### 4. Groups of reactions
This section presents different groups of reactions which are included in the model and can help to get an overview or to order the reactions.
These groups of reactions are interesting in terms of analysing specific aspects of the network.

In [82]:
groups = model2.groups
groupNames = getNames(groups)
groupDf = pd.DataFrame(groupNames, columns=["groupName"])
groupDf.head()

,groupName
0,ATPase
1,"Alanine, aspartate and glutamate metabolism"
2,Alkane metabolism
3,Amino acid metabolism
4,Amino sugar and nucleotide sugar metabolism


### 3. Simulation of a model (FBA)
In the following slot the simulation of growth and its steps will be shown.

#### Workflow:
1. Investigate the environment
2. Investigate the objective function
3. Simulation and result interpretation

As you can see we have a medium with uptake of differnt reaction with 1000 and one reaction (R1070) with 2.43. This reaction (R1070) is the glucose exchange reaction, which means that the glucose uptake of this cell is set to 2.43mmol. This leads to a growth rate of 0.283.

In [86]:
# 1. Investigate the environment
print("The model medium looks like this:\n", model2.medium)
# 2. Investigate the objective function
print("The objective function looks like this:\n", model2.objective)
# 3. Simulation and result interpretation
model2.optimize() # glucose uptake of 2.43 leads to a growth rate of 0.283 

The model medium looks like this:
 {'R1016': 1000.0, 'R1030': 1000.0, 'R1070': 2.43, 'R1164': 1000.0, 'R1189': 1000.0, 'R1287': 1000.0, 'R1296': 1000.0, 'R1298': 1000.0, 'R1323': 1000.0, 'R1333': 1000.0, 'R1364': 1000.0}
The objective function looks like this:
 Maximize
1.0*biomass_C - 1.0*biomass_C_reverse_c1d5c


,fluxes,reduced_costs
R1,0.000000,-8.326673e-17
R2,0.000000,-8.066464e-17
R3,0.000000,-3.330746e-03
R4,0.266702,0.000000e+00
R5,0.000000,5.551115e-17
...,...,...
R2281,0.000000,0.000000e+00
R2282,0.000000,0.000000e+00
R2283,0.000000,2.775558e-17
R2284,0.000000,-5.551115e-17


### 4. Adding a pathway to the model
The following section is about adding the abbility to utilize Galactose into Yarrowia lipolytica (iYli21).

#### Workflow for adding a pathway
1. Find informations about the pathway
2. Add metabolites, reactions and genes
3. Testing the pathway 
4. Writing the model

#### 1. Find information about the pathway
##### a. [Galactokinase](https://www.uniprot.org/uniprotkb/Q6CC28/entry)
- formula: alpha-D-galactose + ATP = ADP + alpha-D-galactose 1-phosphate + H+

##### b. [Galactose-1-phosphate uridylyltransferase](https://www.uniprot.org/uniprotkb/Q6C0K3/entry)
- formula: alpha-D-galactose 1-phosphate + UDP-alpha-D-glucose = alpha-D-glucose 1-phosphate + UDP-alpha-D-galactose

##### c. [UDP-glucose-4-Epimerase](https://www.uniprot.org/uniprotkb/Q6C4H2/entry)
- formula: UDP-alpha-D-glucose = UDP-alpha-D-galactose

##### d. External Alpha-D-Galactose, exchange and transport reaction
- formula: / = alpha-D-galactose_external (exchange)
- formula: alpha-D-galactose_external = alpha-D-galactose_internal (transport)

#### 2. Add metabolites, reactions and genes
##### 1. External Alpha-D-Galactose, exchange and transport reaction

In [ ]:
# add external metabolite: 
dGalactose_ex = Metabolite(
    'M_mccs_aDgalex',
    formula='C6H12O6',
    name='alpha-D-galactose',
    compartment='C_ex')
    
# add exchange reaction 
reaction = Reaction('R_mccs_1')
reaction.name = 'alpha-D-galactose exchange'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
reaction.add_metabolites({
    dGalactose_ex: -1.0,
})
model2.add_reactions([reaction])

# add transport
reaction = Reaction('R_mccs_2') 
reaction.name = 'alpha-D-galactose transport'
reaction.lower_bound = -1000.  # Like arabinose transport
reaction.upper_bound = 1000.  # This is the default
alphaDgalactose = model2.metabolites.get_by_id('m544[C_cy]')
reaction.add_metabolites({
    dGalactose_ex: -1.0,
    alphaDgalactose: 1.0,
})
model2.add_reactions([reaction])

##### 2. [Galactokinase](https://www.uniprot.org/uniprotkb/Q6CC28/entry)
alpha-D-galactose + ATP = ADP + alpha-D-galactose 1-phosphate + H+

In [ ]:
## Take metabolites as they are
alphaDgalactose = model2.metabolites.get_by_id('m544[C_cy]')
alphaDgalactose1phosphate = model2.metabolites.get_by_id('m545[C_cy]')
UDPalphaDglucose = model2.metabolites.get_by_id('m8[C_cy]')
alphaDglucose1phosphate = model2.metabolites.get_by_id('m589[C_cy]')
UDPalphaDgalactose = model2.metabolites.get_by_id('m546[C_cy]')
ATP_cy = model2.metabolites.get_by_id('m141[C_cy]')
ADP_cy = model2.metabolites.get_by_id('m143[C_cy]')
h_plus = model2.metabolites.get_by_id('m10[C_cy]')

### Galactokinase
## Reaction alpha-D-galactose + ATP => alpha-D-galactose-1-phosphate + ADP

reaction = Reaction('R_mccs_3') 
reaction.name = 'Galactokinase'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
reaction.add_metabolites({
    alphaDgalactose: -1.0,
    ATP_cy: -1.0,
    ADP_cy: 1.0,
    alphaDgalactose1phosphate: 1.0,
    h_plus: 1.0
})
reaction.reaction
model2.add_reactions([reaction])

##### 3. [Galactose-1-phosphate uridylyltransferase](https://www.uniprot.org/uniprotkb/Q6C0K3/entry)
alpha-D-galactose 1-phosphate + UDP-alpha-D-glucose = alpha-D-glucose 1-phosphate + UDP-alpha-D-galactose

In [ ]:
### Galactose-1-phosphate uridylyltransferase
# alpha-D-galactose 1-phosphate + UDP-alpha-D-glucose = alpha-D-glucose 1-phosphate + UDP-alpha-D-galactose

reaction = Reaction('R_mccs_4') 
reaction.name = 'Galactose-1-phosphate uridylyltransferase'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
reaction.add_metabolites({
    alphaDgalactose1phosphate: -1.0,
    UDPalphaDglucose: -1.0,
    alphaDglucose1phosphate: 1.0,
    UDPalphaDgalactose: 1.0,
})
reaction.reaction
model2.add_reactions([reaction])

##### 4. [UDP-glucose-4-Epimerase](https://www.uniprot.org/uniprotkb/Q6C4H2/entry)
UDP-alpha-D-glucose = UDP-alpha-D-galactose


In [ ]:
### UDP-glucose-4-Epimerase
## UDP-alpha-D-glucose = UDP-alpha-D-galactose

reaction = Reaction('R_mccs_5') 
reaction.name = 'alpha-D-galactose exchange'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default
reaction.add_metabolites({
    UDPalphaDgalactose: -1.0,
    UDPalphaDglucose: 1.0,
})
reaction.reaction
model2.add_reactions([reaction])

#### 3. Testing the pathway 

##### a. Set the other carbon uptakes equal to zero
##### b. Set galactose uptake to a realistic value (2.43mmol/L)
##### c. Simulate growth using FBA (with biomass as objective)

In [ ]:
# a. Set all the other influxes to zero (carbon sources)
# Get glucse flux bounds and set them to zero
glucoseReac = model2.reactions.get_by_id('R1070')
print('glucose reaction bounds: ', glucoseReac.bounds) 
# set to null
glucoseReac.bounds = (0.0,0.0)

# b. galactose: R_mccs_1
galactoseReac = model2.reactions.get_by_id('R_mccs_1')
print('galactose reaction bounds: ', galactoseReac.bounds)
# set the lower bound to -0,015795
galactoseReac.bounds = (-2.43, 0.0)
print('galactose reaction bounds: ', galactoseReac.bounds)

# c. FBA (with biomass as objective)
print(model2.medium)
galactose = model2.optimize() # optimized value 0.283
print(galactose)
galactoseReac.bounds = (0.0,0.0)

# d. compare to anouter carbon source (glucose)
glucoseReac.bounds = (-2.43,0.0)
print('glucose reaction bounds: ', glucoseReac.bounds) 
print(model2.medium)
glucose = model2.optimize() # optimized value 0.283
print(glucose)

glucose reaction bounds:  (-2.43, 1000.0)
galactose reaction bounds:  (0.0, 1000.0)
galactose reaction bounds:  (-2.43, 0.0)
{'R1016': 1000.0, 'R1030': 1000.0, 'R1164': 1000.0, 'R1189': 1000.0, 'R1287': 1000.0, 'R1296': 1000.0, 'R1298': 1000.0, 'R1323': 1000.0, 'R1333': 1000.0, 'R1364': 1000.0, 'R_mccs_1': 2.43}
<Solution 0.283 at 0x7f9e7ad6c280>
glucose reaction bounds:  (-2.43, 0.0)
{'R1016': 1000.0, 'R1030': 1000.0, 'R1070': 2.43, 'R1164': 1000.0, 'R1189': 1000.0, 'R1287': 1000.0, 'R1296': 1000.0, 'R1298': 1000.0, 'R1323': 1000.0, 'R1333': 1000.0, 'R1364': 1000.0}
<Solution 0.283 at 0x7f9e7ad6c3a0>


The result means, that with the uptake of 2.43 mmol of galactose the growth is equal to the growth with uptake of 2.43 mmol glucose. This is possible because both are sugars with six carbon atoms.

#### 4. Write model into sbml format

In [ ]:
### write model in sbml format
write_sbml_model(model2, outdir + extendedModelName)